![Unidata organization logo](https://github.com/Unidata/metpy-ams-2023/raw/main/logos/unidata_logo_horizontal.png)

# Radar Data

## MetPy for your Data: Analyzing Meteorological Observations in Python

**About this Notebook**:

When working with radar data in Python, we'll be going beyond just MetPy. This notebook will introduce Py-ART (another package in the Python ecosystem in the atmospheric sciences) and use it to read Level 2 NEXRAD data from AWS S3. MetPy, as well as several other packages used so far in this course, will still be in use as well.

## Table of Contents <a class="anchor" id="top"></a>

* [Activity 0: Import required packages](#step0)
* [Activity 1: Exploring the NEXRAD level 2 archive on AWS S3](#step1)
* [Activity 2: Loading a level 2 file with Py-ART](#step2)
* [Activity 3: Plotting radar data with matplotlib](#step3)
    * [Step 3a: Plotting a single moment](#step3a)
    * [Step 3b: Plotting mulitple scans/moments](#step3b)
* [Wrap-Up](#wrapup)

## Activity 0: Import required packages <a class="anchor" id="step0"></a>
[Top](#top)

First, we need to import all our required packages. In this notebook we're primarily working with:

- Py-ART, for reading NEXRAD level 2 data
- s3fs, for interacting with an S3 bucket in a filesystem-like way
- matplotlib, for plotting
- cartopy, for geographically-aware plotting
- MetPy, for county outlines

In [1]:
## CELL 0A
## INSTRUCTIONS: Run this cell

from datetime import datetime
import re
import warnings

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from metpy.plots import USCOUNTIES
import numpy as np
import pyart
import s3fs

# Suppress warnings from cartopy with shapely 2.0
warnings.filterwarnings('ignore', '.*geom_factory.*', DeprecationWarning)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
/home/jthielen/mambaforge/envs/metpy-ams-2023/lib/python3.11/site-packages/pyart/io/nexrad_level3.py:72: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from xdrlib import Unpacker
/home/jthielen/mambaforge/envs/metpy-ams-2023/lib/python3.11/site-packages/botocore/utils.py:15: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi
/home/jthielen/mambaforge/envs/metpy-ams-2023/lib/python3.11/site-packages/botocore/httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


## Activity 1: Exploring the NEXRAD level 2 archive on AWS S3<a class="anchor" id="step1"></a>
[Top](#top)

When it comes to operational radar data in the U.S. (both archive and real-time), [the NEXRAD collections on AWS S3](https://registry.opendata.aws/noaa-nexrad/) are often the place to turn to first. Here, we will be focusing on *Level 2* NEXRAD radar data, rather than the more highly-processed and reduced-resolution Level 3 data available from NEXRAD sites ([for more info, see this NCEI info page](https://www.ncei.noaa.gov/products/radar/next-generation-weather-radar)), or any non-NEXRAD formats.

While S3 is indeed something called an *[object store](https://en.wikipedia.org/wiki/Object_storage)*, rather than a traditional file system or a web-based API, this isn't something that need cause concern. Many utilities exist to interact with object stores as if they were simple file systems (we will be using one below) and many buckets expose web interfaces through which to browse files. The `noaa-nexrad-level2` S3 bucket used in this notebook can be [explored here](https://s3.amazonaws.com/noaa-nexrad-level2/index.html), however, we can also simply explore the contents via code!

Using `s3fs`, we can look at the top-level contents/directories in this bucket:

In [2]:
## CELL 1A
## INSTRUCTIONS: Run this cell

fs = s3fs.S3FileSystem(anon=True)
fs.ls('noaa-nexrad-level2')

['noaa-nexrad-level2/1970',
 'noaa-nexrad-level2/1991',
 'noaa-nexrad-level2/1992',
 'noaa-nexrad-level2/1993',
 'noaa-nexrad-level2/1994',
 'noaa-nexrad-level2/1995',
 'noaa-nexrad-level2/1996',
 'noaa-nexrad-level2/1997',
 'noaa-nexrad-level2/1998',
 'noaa-nexrad-level2/1999',
 'noaa-nexrad-level2/2000',
 'noaa-nexrad-level2/2001',
 'noaa-nexrad-level2/2002',
 'noaa-nexrad-level2/2003',
 'noaa-nexrad-level2/2004',
 'noaa-nexrad-level2/2005',
 'noaa-nexrad-level2/2006',
 'noaa-nexrad-level2/2007',
 'noaa-nexrad-level2/2008',
 'noaa-nexrad-level2/2009',
 'noaa-nexrad-level2/2010',
 'noaa-nexrad-level2/2011',
 'noaa-nexrad-level2/2012',
 'noaa-nexrad-level2/2013',
 'noaa-nexrad-level2/2014',
 'noaa-nexrad-level2/2015',
 'noaa-nexrad-level2/2016',
 'noaa-nexrad-level2/2017',
 'noaa-nexrad-level2/2018',
 'noaa-nexrad-level2/2019',
 'noaa-nexrad-level2/2020',
 'noaa-nexrad-level2/2021',
 'noaa-nexrad-level2/2022',
 'noaa-nexrad-level2/2023',
 'noaa-nexrad-level2/index.html']

In this Level 2 bucket, the *object keys* that specify where in the bucket our data file is located look like paths structured in the following way:

`noaa-nexrad-level2/{year}/{month}/{day}/{site}/{filename}`

where `{filename}` typically looks like:

* `{site}{yr}{mo}{dy}_{hr}{min}{sec}_V06`, for more recent data
* `{site}{yr}{mo}{dy}_{hr}{min}{sec}_V06.gz`, for slightly less recent data
* `{site}{yr}{mo}{dy}_{hr}{min}{sec}_V03.gz` or `{site}{yr}{mo}{dy}_{hr}{min}{sec}.gz`, for older data

While NEXRAD volumes are collected in a recurring fashion, their collection times are not regular/standardized. And so, to find the radar data valid for any given time of interest, we cannot simply know the right file name to request, and instead need to query our file collection for files that are proximate in time.

For example, here we present some of the radar files available from KFTG from today.

In [3]:
## CELL 1B
## INSTRUCTIONS: Run this cell

now = datetime.now()
file_keys = fs.ls(f'noaa-nexrad-level2/{now:%Y}/{now:%m}/{now:%d}/KFTG/')

print(len(file_keys))
print(file_keys[-5:])

84
['noaa-nexrad-level2/2023/01/08/KFTG/KFTG20230108_071119_V06', 'noaa-nexrad-level2/2023/01/08/KFTG/KFTG20230108_071813_V06', 'noaa-nexrad-level2/2023/01/08/KFTG/KFTG20230108_072515_V06', 'noaa-nexrad-level2/2023/01/08/KFTG/KFTG20230108_073216_V06', 'noaa-nexrad-level2/2023/01/08/KFTG/KFTG20230108_073916_V06']


While one could certainly browse through the available files manually, it would be much more effective to do so with some code that searches the possible matching volume files for that closest to a given time.

For example, still at KFTG, let's try to find the scan closest to 2013-09-12 01:00 UTC:

In [4]:
## CELL 1C
## INSTRUCTIONS: Run this cell

site = 'KFTG'
target = datetime(2013, 9, 12, 1, 0)
file_keys_on_day = fs.ls(f'noaa-nexrad-level2/{target:%Y}/{target:%m}/{target:%d}/{site}/')

def key_to_time_difference(key, target):
    # Search for the filename in the key using a regex pattern
    if dt_group := re.search(r'[A-Z]{4}(\d{8}_\d{6})', key):
        return (target - datetime.strptime(dt_group.group(1), '%Y%m%d_%H%M%S')).total_seconds()
    else:
        return np.inf
    
nearest = min(file_keys_on_day, key=lambda k: abs(key_to_time_difference(k, target)))

print(nearest)

noaa-nexrad-level2/2013/09/12/KFTG/KFTG20130912_010204_V06.gz


<div class="admonition alert alert-warning">

<p class="admonition-title" style="font-weight:bold">Activity: Find Your Own Level 2 File</p>

Now, find a radar file yourself from this S3 bucket for a meteorological event that you'd be interested in exploring today. Choose a site and a target datetime, and adapt the code above (or any other approach you wish to implement within the short time period for this brief activity) to obtain the object key (file path) for the nearest Level 2 volume.
    
If you have a particular event and location in mind, but don't know what NEXRAD site to use, [this map](https://www.roc.noaa.gov/WSR88D/Maps.aspx) may be helpful. If you don't have an event in mind, feel free to use one of the suggestions below or ask a neighbor for an idea:
    
- August 2020 Derecho - KDMX, 2020-08-10 16:00 Z
- 2011 Super Outbreak - KBMX, 2011-04-27 20:00 Z
- Hurricane Ian - KTBW, 2022-09-28 21:00 Z

</div>

In [ ]:
# ACTIVITY: Find Your Own File
# Adapt prior code or create your own to find a Level 2 file for a time and NEXRAD site of interest

## Activity 2: Loading a level 2 file with Py-ART<a class="anchor" id="step2"></a>
[Top](#top)

Now that we know how to find a level 2 file on the S3 bucket, now we can do something with it!

For this, we'll be using Py-ART, a Python package for radar data released by ARM-DOW. Recent versions integrate with `fsspec`/`s3fs` (like we used earlier) to include easy support for S3 file access. All we need to do to build upon what we've previously done to find a file of interest is add the proper protocol specification and pass it to Py-ART's reader:

In [ ]:
## CELL 2A
## INSTRUCTIONS: Run this cell
## OPTIONAL INSTRUCTIONS: Modify variable "nearest" to what your file is represented with
##     from the prior activity, if changed.

radar = pyart.io.read_nexrad_archive(f's3://{nearest}')
radar

The data model (currently) used by Py-ART is a bespoke collection of `dict`s representing radar data fields as well as metadata. Using `.info()` on the object output by the reader, we can preview these fields and view all the metadata:

In [ ]:
## CELL 2B
## INSTRUCTIONS: Run this cell

radar.info()

As a quick demonstration of how the data in these collections can be accessed, let's look at a quick plot of the elevation angle of the radar beam over the course of this volume:

In [ ]:
## CELL 2C
## INSTRUCTIONS: Run this cell

plt.plot(radar.elevation['data'])

## Activity 3: Plotting radar data with matplotlib<a class="anchor" id="step3"></a>
[Top](#top)

We now have a Level 2 radar file loaded into memory and can explore its metadata. For the last section, let's actually look through the data!

Plotting radar data merits some special concern because these data exist in *radar-centered polar coordinates*: data cells are separated by grid spacings of azimuth, range, and elevation, rather than x/y distance or latitude/longitude. And so, we will need to use a mesh-capable plotting routine to represent these non-rectangular data cells. Thankfully, the details of *coordinate system transformations* can be handled easily between cartopy and Py-ART, so our plotting code can remain relatively precise.

### Activity 3a: Plotting radar data with matplotlib<a class="anchor" id="step3a"></a>
[Top](#top)

Below is a simple example of plotting such polar coordinate radar data. Note the following:

- We are using an *Azimuthal Equidistant* CRS, so that we can validly geo-reference our data based on x and y distances from the radar location.
- We are choosing a *CVD-friendly* colormap from Py-ART. For more options from Py-ART, see [this section of their docs](https://arm-doe.github.io/pyart/examples/plotting/plot_choose_a_colormap.html).
- Py-ART separates different collections of rays of data at (approximately) the same elevation into *sweeps*, and so we use sweep-based methods to extract 2D data for plotting.
- Py-ART's `get_gate_x_y_z` method calculates the x and y (and z, which we don't use in this example) distances from the radar to the data cell center. Given that we chose an Azimuthal Equidistant CRS, these are exactly the projection coordinates that cartopy needs to plot our data on a map.

In [ ]:
## CELL 3A
## INSTRUCTIONS: Run this cell

crs = ccrs.AzimuthalEquidistant(
    central_longitude=radar.longitude['data'].item(),
    central_latitude=radar.latitude['data'].item()
)
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': crs})
sweep = radar.extract_sweeps([0])
x, y, z = sweep.get_gate_x_y_z(0)

ax.pcolormesh(x, y, sweep.fields['reflectivity']['data'], cmap='pyart_ChaseSpectral', vmin=-10, vmax=70)
ax.add_feature(USCOUNTIES, linewidth=0.5)
ax.set_extent([-2e5, 1e5, -1e5, 2e5], crs=crs)
ax.set_title(nearest.split('/')[-1])

plt.show()

<div class="admonition alert alert-warning">

<p class="admonition-title" style="font-weight:bold">Activity: Make an Improved Plot of a Single Field</p>

This previous plot was initially simplified. For a useful scientific figure, there are several things we'd want to see improved over this previous plot, such as adding colorbar and using a more meaningful/descriptive figure title. Choose one (or any number) of improvements to this figure to make an improved plot of a single field of radar data.
    
Also, feel free to explore different fields or sweeps within your chosen volume!
</div>

In [ ]:
# ACTIVITY: Plot an Improved Plot of a Single Field
# Create your plot code below

### Activity 3b: Plotting mulitple scans/moments<a class="anchor" id="step3b"></a>
[Top](#top)

To end this section, let's go from just viewing a single plot of radar data to creating a figure to compare several different components of interest, such as:

- multiple different sweeps,
- multiple different fields,
- or even multiple different times (i.e., radar files)

To get you started, see below for one way multiple sweeps could be displayed:

In [ ]:
## CELL 3B
## INSTRUCTIONS: Run this cell

crs = ccrs.AzimuthalEquidistant(
    central_longitude=radar.longitude['data'].item(),
    central_latitude=radar.latitude['data'].item()
)
fig, axes = plt.subplots(2, 2, figsize=(10, 10), subplot_kw={'projection': crs})

for i, ax_row in enumerate(axes):
    for j, ax in enumerate(ax_row):
        sweep_id = 4 * i + 2 * j
        sweep = radar.extract_sweeps([sweep_id])
        x, y, z = sweep.get_gate_x_y_z(0)

        ax.pcolormesh(x, y, sweep.fields['reflectivity']['data'], cmap='pyart_ChaseSpectral', vmin=-10, vmax=70)
        ax.add_feature(USCOUNTIES, linewidth=0.5)
        ax.set_extent([-2e5, 1e5, -1e5, 2e5], crs=crs)
        ax.set_title(f"{sweep.elevation['data'].mean():0.2f}\u00B0 Elevation")
        
fig.suptitle(nearest.split('/')[-1])

plt.show()

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Activity: Make a Multi-Panel Figure</p>
    For your event of interest and the radar data you have, get creative with comparing several different aspects of the data using a multi-panel figure!
</div>

In [ ]:
# ACTIVITY: Plot a Multi-Panel Figure
# Create your plot code below

## Wrap-Up <a class="anchor" id="wrapup"></a>

By working through this notebook, you've learned the basics of working with Level 2 radar data in Python, and could take what you learned here to plot any number of available fields from any sweep from such data.

If this area of data exploration interests you, the following resources may be of interest to you in learning more:

- [Py-ART Example Gallery](https://arm-doe.github.io/pyart/examples/index.html) and [Docs](https://arm-doe.github.io/pyart/index.html)
- [xradar](https://github.com/openradar/xradar), which brings radar data into xarray in the fashion of the CfRadial2 standards
    - NEXRAD support is still under development and not yet available, but should be soon in early 2023
- Learn more about other forms of radar data, such as [NEXRAD Level 3](https://www.ncei.noaa.gov/products/radar/next-generation-weather-radar) or [MRMS](https://www.nssl.noaa.gov/projects/mrms/)